# Nguồn tham khảo:
- Fast R-CNN (Paper)
- https://www.mihaileric.com/posts/fast-object-detection-with-fast-rcnn/ (Hình ảnh rõ ràng)
- https://d2l.ai/chapter_computer-vision/rcnn.html
- https://nttuan8.com/bai-11-object-detection-voi-faster-r-cnn/#Fast_RCNN
- https://github.com/Liu-Yicheng/Fast-RCNN
- https://github.com/mitmul/chainer-fast-rcnn/tree/master/models
- https://lilianweng.github.io/lil-log/2017/12/31/object-recognition-for-dummies-part-3.html#fast-r-cnn + FASTER RCNN
- https://www.phamduytung.com/blog/2018-12-05-what-do-we-learn-from-object-detection-p1/ (Hay) + FASTER  RCNN
- https://www.facebook.com/notes/deep-learning-v%C3%A0-%E1%BB%A9ng-d%E1%BB%A5ng/object-detection-t%E1%BB%AB-r-cnn-%C4%91%E1%BA%BFn-faster-r-cnn/223242788272623/
- https://phamdinhkhanh.github.io/2019/09/29/OverviewObjectDetection.html#42-fast-r-cnn-2015 (Giới thiệu chung về Computer vision và Object Detection) + FASTER RCNN
- https://dothanhblog.wordpress.com/2019/12/29/overview-object-detection-ml/ (chủ yếu Faster RCNN)

# 0. Abstract
Paper này đề xuất một phương pháp Fast Region-based Convolutional Network (Fast R-CNN) cho object detection. Fast R-CNN xây dựng trên những nghiên cứu trước để phát hiện đối tượng một cách hiệu quả sử dụng mạng deep convolution. 
So với các nghiên cứu trước, Fast R-CNN có 1 số cải tiến để training hiệu quả và testing tốc độ đồng thời cũng tăng độ chính xác. Fast R-CNN train mạng rất sâu VGG16 và nhanh hơn 9 lần R-CNN, tốc độ test nhanh hơn 123 lần, và đạt một mAP cao trên PASCAL VOC 2012. so sánh với SPPnet, Fast R-CNN train VGG16 nhanh hơn 3 lần, test nhanh hơn 10 lần và độ chính xác cao hơn. Fast R-CNN thực thi trên python và C++ (Caffe) và có sẵn dưới dạng mã nguồn mở MIT license tại https://github.com/rbgirshick/fast-rcnn

# 1.Introduction
Gần đây, deep ConvNets được cải thiện độ chính xác đáng kể trong bài toán phân lớp ảnh và phát hiện đối tượng. So với phân loại ảnh, phát hiện đối tượng là thách thức lớn hơn yêu cầu các phương pháp phức tạp hơn để giải quyết. Do sự phức tạp này, cách tiếp cận hiện nay train model ở nhiều giai đoạn là chậm.

Phức tạp phát sinh vì phát hiện đối tượng yêu cầu chính xác vị trí các đối tượng, tạo 2 thách thức chính. Đầu tiên, nhiều đối tượng đề xuất phải xử lý. Thứ 2, các đối tượng đề xuất là thô và cần được tinh chỉnh để chính xác. Giải quyết các vấn đề này thường ảnh hưởng tốc độ, độ chính xác...

Trong paper này, chúng tôi tinh giản quá trình training cho state-of-the-air ConvNet-based object detectors. Chúng tôi đưa ra một thuật toán training single-state mà cùng học cách đề xuất đối tượng và tinh chỉnh không gian đối tượng.

Kết quả phương pháp có thể train một mạng deep detection (VGG16 nhanh hơn 9 lần R-CNN và 3 lần SPPnet). Tại mỗi lần chạy, mạng detection xử lý các hình ảnh trong 0.3s (bao gồm thời gian object proposal) trong khi độ chính xác trên PASCAL VOC 2012 với mAP đạt 66% (vs 62% cho R-CNN)

## 1.1 R-CNN and SPPnet
R-CNN xuất sắc trong bài toán phát hiện đối tượng với độ chính xác cao sử dụng deep ConvNet để phân lớp các object proposal. Tuy nhiên, nó có 1 số nhược điểm đáng chú ý sau:
1. **Training qua nhiều giai đoạn:** Đầu tiên R-CNN fine-tuning một ConvNet trên object proposal sử dụng log loss. Rồi nó fit SVM tới các đặc trưng ConvNet. Các SVM như là các trình phát hiện đối tượng, thay thế các class softmax đã học ở fine-tuning. Trong quá trình thứ 3, bouding-box regressor được học.
2. **Training tốn kém không gian và thời gian:** Với quá trình training SVM và bouding-box regressor, các đặc trưng được trích xuất từ mỗi object proposal trong mỗi hình ảnh và lưu vào đĩa. Với mạng rất sâu, như VGG16, quá trình này cần 2.5 ngày GPT cho 5000 ảnh của tập trainval VOC07. Các đặc trưng yêu cầu hàng trăm GB lưu trữ.
3. **Object detection là chậm:** Tại mỗi lần test, các đặc trưng được trích xuất từ mỗi object proposal trong mỗi ảnh test. Detection với BGG16 cần 47s/ ảnh (trên GPU)

R-CNN là chậm bởi vì nó thực hiện một ConvNet qua mỗi object proposal, không chia sẻ tính toán.

**BỐC PHÉT VỀ SPP**

## 1.2 Contributions
Chúng tôi đề xuất một thuật toán training mới sữa chữa nhược điểm của R-CNN và SPPnet , đồng thời cải thiện tốc độ và độ chính xác. Chúng tôi gọi phương pháp này là `Fast R-CNN` bởi vì nó nhanh hơn cho cả quá trình train và test. Phương pháp Fast R-CNN có 1 số ưu điểm:
1. Chất lượng phát hiện (mAP) cao hơn R-CNN, SPPnet
2. Training single-stage, sử dụng multi-task loss
3. Training có thể cập nhật tất cả các layer mạng
4. Không lưu trữ ra đĩa.

# 2. Fast R-CNN architecture and training
**HÌNH 1** minh hoạ kiến trúc mạng Fast R-CNN. Một mạng Fast R-CNN lấy một input là một ảnh và một tập các object proposal. Mạng đầu tiên sẽ xử lý hình ảnh với conv và max pooling layer để cho ra một conv feature map. Rồi với mỗi object proposal một region of interest (RoI) pooling layer trích xuất 1 vector đặc trưng độ dài cố định. Mỗi vector đặc trưng được fed vào một chuỗi FC layer mà cuối cùng nhánh đi qua 2 sibling output lyaer: một lớp tạo ra các ước tính xác suất softmax trên các lớp đối tượng K cộng với một lớp nền nền bắt tất cả và một lớp khác tạo ra bốn số có giá trị thực cho mỗi lớp đối tượng K. Mỗi bộ 4 giá trị mã hóa các vị trí hộp giới hạn được tinh chỉnh cho một trong các lớp K.

## 2.1 The RoI pooling layer
Layer RoI pooling sử dụng max pooling để chuyển các đặc trưng bên trong region of interest thành một bản đồ đặc trưng nhỏ (small feature map) với 1 phạm vi cố định HxW (ví dụ 7x7) với H và W là siêu tham só độc lập của bất kỳ RoI nào. Trong paper này, một Roi là một hình chữ nhật qua một conv feature map. Mỗi RoI được định nghĩa bởi bộ 4 số (r, c, h, w) tương ứng với toạ độ góc trên trái của nó (r,c) và chiều cao và chiều rộng (h,w).

RoI max poolinbg làm việc bằng cách chia window hxw RoI thành các lưới sub-window HxW kích thước khoảng h/W x w/W và rồi max-pooling giá trị mỗi sub-window thành các cell lưới đầu ra tương ứng. Pooling áp dụng độc lập cho mỗi kênh feature map, như một max pooling chuẩn. Lớp ROI chỉ đơn giản là
đặc biệt hợp cụ thể của kim tự tháp không gian lớp tổng hợp được sử dụng trong SPPnets trong đó chỉ có một mức kim tự tháp. Chúng tôi sử dụng pooling sub-window tổng hợp trong 11.

## 2.2 Initializing from pre-trained networks
## 2.3 Fine-tuning for detection
Training toàn weight của mạng với lan truyền ngược là quan trọng. First, let’s elucidate why SPPnet is unable to update weights below the spatial pyramid pooling layer.
....
Chúng tôi đề xuất một phương pháp huấn luyện hiệu quả ưu điểm ở việc chia sẻ đặc trưng trong quá trình hauasn luyện. Trong Fast R-CNN, SGD mini-batches lấy mẫu phân cấp, đầu tiên N mẫu hình ảnh và rồi R/N mẫu RóI từ mỗi ảnh. 
...
**Multi-task loss:** Một mạng Fast R-CNN có 2 output layer. Output đầu tiên là phân phối (trên mỗi RoI), $p=(p_0, ..., p_K)$ trên K+1 thể loại. Thường $p$ được tính toán bởi hàm softmax trên K+1 đầu ra của lớp fully connected. Lớp output thứ 2 bouding-box regression dự đoán $t^k=(t_x^k, t_y^k, t_w^k, t_h^k)$, với mỗi lớp đối tượng K. 

Mỗi training RoI đã được gán nhãn với một lớp ground-truth $u$ và một ground-truth bouding box regression mục tiêu $v$. Chúng tôi sử dụng một hàm loss multi-task $L$ trên mỗi nhãn RoI để cả train cho phân lớp và bouding-box regression:
$$L(p, u, t^u, v) = L_{cls}(p,u) + \lambda[u \ge 1] L_{loc}(t^u, v)$$
trong đó $L_{cls}(p,u)=-log p_u$ là log loss cho class đúng u.

Hàm loss thứ 2 là $L_{loc}$ định nghĩa qua một tupple bouding-box regression mục tiêu cho lớp u, $v=(v_x, v_y, v_w, v_h)$, và mọt tuple dự đoán $t^u=(t^u_x, t^u_y, t^u_w, t^u_h)$ cho class u. hàm $[g \ge 1]$ bằng 1 khi $u \ge 1$ và 0 trong các trường hợp còn lại, theo quy ước tất cả các lớp nền nhận nhãn $u=0$. Đối với Rois nền không có khái niệm về ground-truth bouding box và  và vì thế $L_{loc}$ là ignored. Với bouding box regression, chúng tôi sử dụng loss:
$$L_{loc}(t^u, v) = \sum_{i \in \{x,y,w,h\}} smooth_{L1} (t^u_i - v_i)$$

trong đó:
$smooth_{L1}(x)=0.5x^2$ nếu $|x| <1$ và bằng $|x|-0.5$ các trường hợp khác.

là một L1 mất mạnh mẽ đó là ít nhạy cảm với giá trị ngoại biên hơn
mất L2 sử dụng trong R-CNN và SPPnet. Các siêu tham số λ trong phương trình. 1 điều khiển sự cân bằng giữa hai lỗ nhiệm vụ. (các thí nghiệm cài là 1)


# 3. Fast R-CNN detection

# 4. Main result

# 5. Design evaluation

# 6. Conclusion

# Nguồn: 
https://www.facebook.com/notes/deep-learning-v%C3%A0-%E1%BB%A9ng-d%E1%BB%A5ng/object-detection-t%E1%BB%AB-r-cnn-%C4%91%E1%BA%BFn-faster-r-cnn/223242788272623/

Như chúng ta đã biết object detection bao gồm 2 nhiệm vụ chính là Classifier và Localization. Trong đó nhiệm vụ có vẻ khó khăn hơn là Localization. Trước khi deep learning phát triển như hiện nay, trong computer vision người ta detection object qua 2 giai đoạn. Đầu tiên là trích xuất feature từ hog,lbp,sift sau đó dùng các thuật toán trong machine learning như SVM để classifier. Bước tiếp theo là detection object trên ảnh lớn thì người ta sẽ dùng 1 window search trên toàn bộ bức ảnh sau đó dùng model đã classifier để phân lớp object. Các model này có ưu điểm là thời gian build model tương đối nhanh, cần ít dữ liệu . Nhược điểm là độ chính xác không cao và thời gian predict rất lâu nên khó có thể dùng trong real time.

Với tộc độ phát triển như hiện nay , dữ liệu của chúng ta ngày càng nhiều và các bài toán bắt đầu khó dần lên nên những model truyền thống tỏ ra kém hiệu quả. Các feature lấy ra từ computer vision truyền thống như hog,sift,lbp là những shadow feature nó chỉ lấy được những feature trên bề mặt nổi image mà thôi, do đó những bài toán như classifer con chó hay con mèo thì những feature này làm việc tương đối hiệu quả , nhưng nâng cấp bài toán lên đó là classifier con bull dog hay con béc rê thì những feature này làm việc kém hiệu quả. Cũng dễ hiểu vì cùng 1 class là dog thì những feature này tương đối giống nhau nên khó có thể phân biệt được con này con kia. Chính vì thế ta cần những feature sâu hơn, những feature mà nó ẩn ở trong image mà ta khó có thể quan sát được bằng mắt thường để phân biệt dog này hay dog kia. Trong deep learning , object detection nền tảng cơ bản là dựa trên mạng CNN để lấy những deep feature bằng cách đưa qua nhiều layer khác nhau feature được extract sâu hơn sau đó được đưa vào classifier và regression box. Hai hướng tiếp cận chính trong deep learning là :
* Chia image ra thành những grid cell SxS . Mỗi cell được coi như region proposal giúp giảm thời gian và chi phí tính toán thay vì sử dụng trực tiếp image ( model SSD,YOLO)
* Tìm những region proposal có nhiều khẳn năng chứa object nhất sử dụng selective search hay RPN ( model R-CNN,Fast R-CNN, Faster R-CNN)

Mô hình RCNN:
![rcnn](./images/rcnn.png)

R-CNN là viết tắt của “Region-based Convolutional Neural Networks”. Ý tưởng chính của nó gồm 2 phần : Đầu tiên là dùng selective search tìm các region of interest (roi) trên image để tạo ra các bounding box mà có xác suất cao là có object. Sau đó dùng CNN để lấy feature từ các region này để classifier và regression  box. Các bước thực hiện như sau :
1. Từ input image ta dùng selective search để lấy cái region proposal .Selective Search nó hoạt động bằng cách là đầu tiên tạo ra các seed là các region segementation trên image(trong skimage họ dùng Felsenszwalb’s efficient graph based image segmentation) Các region sau đó sẽ được merger lại với nhau bằng cách tính độ tương đồng về color,shape,textuture… Cuối cùng ta vẽ bounding box cho từng region.Mỗi image người ta sẽ lấy tầm 2k region proposal.
2. Tiếp đến các region proposal sẽ được swarped(crop) để fix size(vì một số mạng như VGG yêu cầu size input là cố định,hơn nữa ta cần feature output same size). Sau đó dùng CNN( VGG,Alexnet) để lấy feature.

## SPPnet
![spp](./images/spp.png)

Thay vì feed forword 2k image qua CNN thì người ta feed forwork qua CNN một lần để lấy feature map (feature map = feature + location). Sau đó dùng selective search để tìm region proposal, rồi project trên feature map để lấy feature tương ứng. Có một vấn đề ở đây là các feature map của region proposal có size khác nhau nên khi đưa qua CNN sẽ có length output khác nhau. Vì vậy người ta sử dụng Spatial paramy pooling layer để  fix size feature.(spp layer hoạt động  cũng tương tự bag of word trong image processing nó sẽ chia feature theo Spatial pyramid và áp dụng max pooling theo từng spatial giúp các feature có size khác nhau thành same size). Phần sau còn lại tương tự như R-CNN

## Fast RCNN
![fast-rcnn](./images/fast-rcnn.png)

Fast R-CNN cải thiện được các nhược điểm của R-CNN bằng cách hợp nhất 3 model độc lập vốn rất chậm chạp. Nó cũng có phần giống SPP-net là dùng CNN để lấy feature map một lần thay vì dùng riêng cho mỗi region proposal. Sau đó những feature này sẽ được đưa qua một Fully connection layer để classifier và regression bounding box. Model này tương đối nhanh chạy 2s/image.
Các bước thức hiện thuật toán :
Dùng pretraining model (VGG,ZF…) để lấy feature map.
1. Sử dụng selective search để lấy region proposal (~2k image). Sau đó project lên feature map để lấy feature tương ứng
2. Feature sẽ được đưa qua ROI pooling để fix size.
3. Cuối cùng Fully connection layer để classifier và regression box

# Nguồn:
https://www.phamduytung.com/blog/2018-12-05-what-do-we-learn-from-object-detection-p1/

# Sliding-window detectors

Bắt đầu từ năm 2012, sau khi mạng AlexNet giành giải nhất cuộc thi 2012 ILSVRC, mọi nghiên cứu về phân lớp dữ liệu đều sử dụng mạng CNN. Kể từ đó đến đây, CNN được coi như là thuật toán thống trị trên mọi publish paper về các bài toán phân lớp đối tượng. Trong khi đó, để nhận dạng 1 đối tượng trong ảnh, các đơn giản nhất là thiết lập một cửa sổ trượt có kích thước là window size trượt từ trái qua phải, từ trên xuống dưới, quét qua toàn bộ bức ảnh. Để phát hiện các đối tượng khác nhau ở các góc nhìn khác nhau, chúng ta sẽ sử dụng cửa sổ trượt có kích thước thay đổi và ảnh đầu vào có kích thước thay đổi.
![sliding-window.gif](./images/sliding-window.gif)
![region-base-various-windowsize.png](./images/region-base-various-windowsize.png)
![region-base-various-windowsize1](./images/region-base-various-windowsize1.jpg)

Dựa vào windowsize, chúng ta có thể cắt tấm hình bự thành các tấm hình nhỏ, sau đó sẽ rescale các phần nhỏ của bức ảnh thành các bức ảnh có kích thước cố định.
![fixed-size-image.jpg](./images/fixed-size-image.jpg)
Các phần của bức ảnh sau đó sẽ được đem qua bộ phân lớp CNN để rút trích các đặc trưng, sau đó sử dụng một hàm phân lớp (như svm, logictic regression) để xác định lớp của bức hình và sử dụng linear regressor để tìm bao đóng của đối tượng.
![sliding-window-detector.png](./images/sliding-window-detector.png)

Cách dễ dàng nhất để cải tiến hiệu năng của phương pháp này là giảm số lượng tấm hình nhỏ xuống (ví dụ tăng kích thước window size). Cách này còn được giang hồ gọi là brute force.

# Selective Search
Thay vì hướng tiếp cận brute force ở trên, chúng ta sử dụng phương pháp region proposal để tạo các region of interest (ROIs) để phát hiện đối tượng. Selective search là một phương pháp nằm trong nhóm region proposal. Trong phương pháp selective search(SS), chúng ta bắt đầu bằng cách xem các pixel là mỗi nhóm, các lần lặp tiếp theo, chúng ta sẽ tính khoảng cách ngữ nghĩa (ví dụ như là màu sắc, cường độ ánh sáng) giữa các nhóm và gom các nhóm có khoảng cách gần nhau về chung 1 nhóm để tìm ra phân vùng có khả năng cao nhất chứa đối tượng (ưu tiên gom những nhóm nhỏ trước).

Như hình bên dưới, dòng đầu tiên, bức ảnh đâu tiên là ta có một vài nhóm nhỏ ở thời điểm X nào đó, ở hình thứ 2 là thực hiện gom nhớm theo cường độ màu sắc của hình số 1, và ở bước cuối cùng, ta thu được hình số 3. Những hình chữ nhật màu xanh ở dòng thứ 2 là những ROIS mô phỏng quá trình gom nhóm để tìm phân vùng có khả năng chứa đối tượng.

![selectivesearch.png](./images/selectivesearch.png)

# Mạng R-CNN
Mạng R-CNN sử dụng phương pháp region proposal để tạo ra khoảng 2000 ROIs. Các vùng sau đó sẽ được rescale theo một kích thước cố định nào đó và được đưa vào mô hình CNN có lớp cuối cùng kà một full conected layer để phân lớp đối tượng và để lọc ra boundary box (bao đóng) của đối tượng.

Mô phỏng việc sử dụng region proposal
![region-proposals-r-cnn.png](./images/region-proposals-r-cnn.png)
Với việc sử dụng ít tấm ảnh nhỏ hơn, và chất lượng của mỗi tấm ảnh nhỏ tốt hơn, Mạng R-CNN chạy nhanh hơn và có độ chính xác cao hơn so với mô hình sử dụng cửa sổ trượt.
# Mạng Fast R-CNN
Trong thực tế, các phân vùng của mạng R-CNN bị chồng lấp một phần / toàn bộ với các phân vùng khác. Do đó, việc huấn luyện và thực thi ( inference ) mạng R-CNN diễn ra khá chậm. Nếu chúng ta có 2000 proposal của mạng R-CNN, chúng ta phải thực hiện 2000 lần việc rút trích đặc trưng, một con số khác lớn.

Thay vì phải rút trích đặc trưng của mỗi proposal, chúng ta có thể dùng CNN rút trích đặc trưng của toàn bộ bức ảnh trước (được feature map), đồng thời rút trích các proposal, lấy các proposal tương ứng trên feature map, rescale và cuối cùng là phân lớp và tìm vị trí của object. Với việc không phải lặp lại 2000 lần việc rút trích đặc trưng, Fast R-CNN giảm thời gian xử lý một cách đáng kể.

Mô phỏngviệc sử dụng propoxal trên feature map và các bước tiếp theo của Fast R-CNN
![region-proposals-fast-r-cnn-network-model.jpg](./images/region-proposals-fast-r-cnn-network-model.jpg)

Với việc không phải lặp đi lặp lại quá trình tìm ra các proposal, tốc độ của thuật toán tăng lên kha khá. Trong thực nghiệm, mô hình Fast R-CNN chạy nhanh hơn gấp 10 lần so với R-CNN trong quá trình huấn luyện. Và nhanh hơn 150 lần trong inferencing.

Một khác biệt lớn nhất của Fast R-CNN là toàn bộ network (feature extractior, classifier, boundary box regressor) có thể huấn luyện end-to end (nghĩa là từ đầu đến cuối) với 2 hàm độ lỗi (loss funtion) khác nhau cùng lúc (classification loss và localization loss). Điều này làm tăng độ chính xác của mô hình.

 ## ROI Pooling
 Vì Fast R-CNN sử dụng full connected layter ở lớp cuối, nên đòi hỏi input của chúng phải có kích thước cố định, nên ta phải resize lại feature về 1 kích thước cố định (do 2000 proposal có kích thước không cố định). Ở đây, các tác giả sử dụng ROI pooling để resize. Thuật toán ở đây được sử dụng như sau:

Giả sử đơn giản là chúng ta có một proposal có kích thước 5x7, và chúng ta cần resize về hình dạng 2x2. Chúng ta xem kỹ hình bên dưới.

![roi-pooling-example.png](./images/roi-pooling-example.png)
Hình ở bên trái là feature map của chúng ta.

Hình số 2, vùng hình chữ nhật xanh là vùng proposal 5x7.

Vì chúng ta cần resize về vùng có kích thước 2x2 (4 phần), nên ta chia vùng proposal 5x7 thành 4 phần (5⁄2 =2 dư 3, vậy có 1 phần là 2, 1 phần là 3. Tương tự 7⁄2 = 3 dư 4, vậy có 1 phần 3, một phần 4. Cuối cùng ta có 4 hình chữ nhật có kích thước tương ứng là 2x3, 2x4, 3x3, 3x4) (Hình số 3).

Hình số 4, từ 4 phần của vùng số 3, ta sẽ lấy giá trị lớn nhất của mỗi vùng.

Vậy là ta thu được feature proposal có kích thước 2x2 rồi.


# Nguồn: 
https://lilianweng.github.io/lil-log/2017/12/31/object-recognition-for-dummies-part-3.html#fast-r-cnn

Để làm cho R-CNN nhanh hơn, Girshick (2015) đã cải tiến quy trình đào tạo bằng cách hợp nhất ba mô hình độc lập thành một khung được đào tạo chung và tăng kết quả tính toán được chia sẻ, được đặt tên là Fast R-CNN.  Thay vì trích xuất các vectơ đặc trưng CNN một cách độc lập cho từng đề xuất khu vực, mô hình này tổng hợp chúng thành một chuyển tiếp CNN trên toàn bộ hình ảnh và các đề xuất khu vực chia sẻ ma trận tính năng này.  Sau đó, ma trận tính năng tương tự được phân nhánh để sử dụng cho việc học phân loại đối tượng và bộ hồi quy hộp giới hạn.  Tóm lại, chia sẻ tính toán tăng tốc R-CNN.
![fast-RCNN-1.png](./images/fast-RCNN-1.png)

## ROI pooling

Nó là một loại max pooling để chuyển đổi các đặc trưng trong vùng được chiếu của hình ảnh có kích thước bất kỳ, hxw, thành 1 cửa sổ nhỏ cố định HxW. Vùng đầu vào được chia thành các lưới HxW, xấp xỉ mọi cửa sổ có kích thước h/H x w/W. Rồi áp dụng max pooling trong mỗi lưới.
![roi-pooling.png](./images/roi-pooling.png)

## Model Workflow

Cách Fast RCNN làm việc được tổng hợp như sau; nhiều bước giống trong R-CNN:
1. Đầu tiên, pre-train một convolutional neural network trên tác vụ phân loại hình ảnh.
2. Đưa ra các vùng (region) bởi selective search (~2k vùng trên mỗi ảnh)
3. Thay đổi pre-train CNN:
    * Thay lớp max pooling cuối cùng của pre-train CNN thành một lớp RoI pooling. Đầu ra của lớp RoI pooling là vector đặc trưng có độ dài cố định của region proposal. Chia sẻ tính toán CNN rất có tính toán, vì nhiều đề xuất khu vực của cùng một hình ảnh rất chồng chéo.
    * Thay thế lớp được kết nối đầy đủ cuối cùng và lớp softmax cuối cùng (lớp K) bằng lớp được kết nối đầy đủ và softmax trên lớp K + 1.
4. Cuối cùng mô hình rẽ nhánh thành 2 layer đầu ra:
    * Công cụ ước tính softmax của các lớp K + 1 (giống như trong R-CNN, +1 là lớp "nền"), xuất ra phân phối xác suất rời rạc cho mỗi RoI.
    * Một mô hình hồi quy hộp giới hạn dự đoán độ lệch so với RoI ban đầu cho mỗi lớp K.

## Loss Function

Mô hình được tối ưu hóa cho một mất mát kết hợp hai nhiệm vụ (phân loại + xác định vị trí vật thể):
- u: True class label, $ u \in 0,1,.., K$; theo quy ước, lớp background $u=0$
- p: Phân phối xác suất rời rạc (per RoI) qua K+1 lớp: $p=(p_0,...,p_K)$, tính toán một softmax qua K+1 đầu ra của FC layer
- v: True bouding box $v=(v_x, v_y, v_w, v_h)$
- $t^u$: Predicted bouding box correction, $t^u = (t^u_x, t^u_y, t^u_w, t^u_h)$

Hàm mất mát bằng tổng của chi phí phân lớp và bouding box regression:
$$\mathcal{L} = \mathcal{L}_\text{cls} + \mathcal{L}_\text{box}$$

Với RoI background, $\mathcal{L}_\text{box}$ bỏ qua với hàm $[u \ge 1]$ được định nghĩa như sau:
$$[u >= 1] = \begin{cases}
    1  & \text{if } u \geq 1\\
    0  & \text{otherwise}
\end{cases}$$

Hàm mất mát tổng thể là:
$$
\begin{align*}
\mathcal{L}(p, u, t^u, v) &= \mathcal{L}_\text{cls} (p, u) +  [u \geq 1] \mathcal{L}_\text{box}(t^u, v) \\
\mathcal{L}_\text{cls}(p, u) &= -\log p_u \\
\mathcal{L}_\text{box}(t^u, v) &= \sum_{i \in \{x, y, w, h\}} L_1^\text{smooth} (t^u_i - v_i)
\end{align*} $$

Mất mát bouding box $\mathcal{L}_{box}$ nên đo sự khác biệt giữa $t^u_i$ và $v_i$ sử dụng hàm mất mát **robust**. Hàm mất mát L1 smooth được đề xuất tại đây và nó được cho là ít nhạy cảm hơn với các ngoại lệ:
$$L_1^\text{smooth}(x) = \begin{cases}
    0.5 x^2             & \text{if } \vert x \vert < 1\\
    \vert x \vert - 0.5 & \text{otherwise}
\end{cases}$$

![l1-smooth.png](./images/l1-smooth.png)

# Speed Bottleneck (Nút cổ chai tốc độ)
Fast R-CNN nhanh hơn nhiều trong cả thời gian đào tạo và thử nghiệm.  Tuy nhiên, sự cải thiện không đáng kể vì các đề xuất khu vực được tạo ra bởi một mô hình khác và điều đó rất tốn kém.